In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Dict, Tuple, Union, Optional
import sys
from pathlib import Path


# Get the parent directory (Datos folder)
current_dir = Path().resolve()  # libretas folder
datos_dir = current_dir.parent  # Datos folder

# Add Datos to path so we can import from Functions
sys.path.insert(0, str(datos_dir))

pd.set_option("display.max_columns", None)


In [4]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import leaguestandingsv3
from datetime import datetime, timezone, timedelta
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard
from Functions.helpfuncs import get_current_season
from Functions.games import *

In [3]:
eastern_conference = {
    'ATL', 'BOS', 'BKN', 'CHA', 'CHI', 'CLE', 'DET', 'IND',
    'MIA', 'MIL', 'NYK', 'ORL', 'PHI', 'TOR', 'WAS'
}

western_conference = {
    'DAL', 'DEN', 'GSW', 'HOU', 'LAC', 'LAL', 'MEM', 'MIN',
    'NOP', 'OKC', 'PHX', 'POR', 'SAC', 'SAS', 'UTA'
}

In [5]:
class Teams():
    def __init__(self, team_abbreviation: str = None, team_id:str = None, team_name:str = None):
        self.current_season = get_current_season()
        provided = [
            team_abbreviation is not None,
            team_id is not None,
            team_name is not None
        ]

        if sum(provided) != 1:
            raise ValueError("You must provide exactly one of: team_abbreviation, team_id, team_name")

        # buscar por el parámetro correcto
        if team_abbreviation is not None:
            self.team_details = self.get_team_details_by_abbreviation(team_abbreviation)

        elif team_id is not None:
            self.team_details = self.get_team_details_by_id(team_id)

        elif team_name is not None:
            self.team_details = self.get_team_details_by_name(team_name)

        # añadir conferencia
        if not self.team_details:
            raise ValueError("Team not found with the provided identifier.")
        abb = self.team_details["abbreviation"]
        self.team_details["conference"] = "East" if abb in eastern_conference else "West"


    def check_valid_season(self, season: str) -> str:
        """Check if the provided season is valid and return formatted season."""
        if not season:
            season = self.current_season
        elif not season.isdigit():
            raise ValueError(f"Invalid season format: {season}. Expected a year in 'YYYY' format.")
        elif int(season) < 1980 or int(season) > datetime.now().year:
            raise ValueError(f"Invalid season year: {season}. Year must be between 1980 and {datetime.now().year}.")
        else:
            season = f"{season}-{str(int(season)+1)[-2:]}"
        
        return season

    def get_all_teams(self) -> pd.DataFrame:
        try:
            all_teams = teams.get_teams()
            teams_df = pd.DataFrame(all_teams)
            return teams_df.to_json(orient="records")
        except Exception as e:
            print(f"Error retrieving all teams: {e}")
            raise e
    
    def get_team_details(self) -> Dict:
        try:
            return self.team_details
        except Exception as e:
            print(f"Error retrieving team details for abbreviation {self.team_abbreviation}: {e}")
            raise e

    def get_team_details_by_abbreviation(self, team_abbreviation:str) -> Dict:
        try:
            team_details = teams.find_team_by_abbreviation(team_abbreviation)
            return team_details
        except Exception as e:
            print(f"Error retrieving team details for abbreviation {team_abbreviation}: {e}")
            raise e
    
    def get_team_details_by_id(self, team_id:int) -> Dict:
        try:
            team_details = teams.find_team_name_by_id(team_id)
            return team_details
        except Exception as e:
            print(f"Error retrieving team details for ID {team_id}: {e}")
            raise e
        
    def get_team_details_by_name(self, team_name:str) -> Dict:
        try:
            team_details = teams.find_teams_by_full_name(team_name)[0]
            return team_details
        except Exception as e:
            print(f"Error retrieving team details for name {team_name}: {e}")
            raise e

    def get_team_roster_per_season(self, season:str = None) -> pd.DataFrame:
        try:
            season = self.check_valid_season(season)

            team_id = self.team_details["id"]
            roster = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
            roster_data = roster.get_data_frames()[0]
            roster_data = roster_data[["PLAYER", "NUM", "POSITION", "HEIGHT", "WEIGHT", "BIRTH_DATE", "AGE", "EXP", "SCHOOL"]]
            return roster_data
        except Exception as e:
            print(f"Error retrieving roster for team {self.team_details['abbreviation']} in season {season}: {e}")
            raise e

    def get_team_last_n_games_played(self, season:str = None, last_n_games:int = 5) -> pd.DataFrame:
        season = self.check_valid_season(season)
        team_details = self.team_details
        team_id = team_details["id"]
        game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)
        game_log = game_log.get_data_frames()[0]

        game_log = game_log[['Game_ID', 'GAME_DATE', 'MATCHUP', 'WL']]
        return game_log[:last_n_games]
    
    def get_team_league_standing(self, season:str = None) -> pd.DataFrame:
        season = self.check_valid_season(season)
        standings = get_current_standings(conference=self.team_details['conference']).reset_index()
        index = int(standings[standings['TeamName'] == self.team_details['nickname']].index[0]) + 1

        return index
    

    def get_team_full_info(self, season:str = None):
        pass



        
    

In [5]:
Teams('LAC').get_team_last_n_games_played()

,Game_ID,GAME_DATE,MATCHUP,WL
0,0022500355,"DEC 06, 2025",LAC @ MIN,L
1,0022500347,"DEC 05, 2025",LAC @ MEM,L
2,0022500327,"DEC 03, 2025",LAC @ ATL,W
3,0022500313,"DEC 01, 2025",LAC @ MIA,L
4,0022500300,"NOV 29, 2025",LAC vs. DAL,L


In [8]:
#Teams().get_all_teams()
#Teams().get_team_details("LAL")
#print(Teams().get_team_roster_per_season("LAL").columns)
#Teams().get_team_roster_per_season("LAL", season=None)
#Teams("LAL").get_team_games_played_per_season(season=None, last_n_games = 5)

lakers = Teams(team_abbreviation="DEN")

#lakers.get_team_league_standing(season="2023")

team_data = lakers.get_team_details()

team_data

{'id': 1610612743,
 'full_name': 'Denver Nuggets',
 'abbreviation': 'DEN',
 'nickname': 'Nuggets',
 'city': 'Denver',
 'state': 'Colorado',
 'year_founded': 1976,
 'conference': 'West'}

In [ ]:
leaderboard = Teams().get_current_standings(conference='East')

In [ ]:
leaderboard[leaderboard['Conference'] == 'West']

,TeamID,TeamCity,TeamName,Conference,WINS,LOSSES,PlayoffRank,Record,HOME,ROAD,L10
0,1610612760,Oklahoma City,Thunder,West,18,1,1,18-1,9-0,9-1,10-0
3,1610612747,Los Angeles,Lakers,West,13,4,2,13-4,5-2,8-2,8-2
5,1610612743,Denver,Nuggets,West,13,4,3,13-4,6-2,7-2,8-2
6,1610612745,Houston,Rockets,West,12,4,4,12-4,5-2,7-2,8-2
9,1610612759,San Antonio,Spurs,West,12,5,5,12-5,8-2,4-3,7-3
10,1610612756,Phoenix,Suns,West,12,7,6,12-7,8-3,4-4,8-2
13,1610612750,Minnesota,Timberwolves,West,10,8,7,10-8,5-3,5-5,6-4
15,1610612744,Golden State,Warriors,West,10,10,8,10-10,6-2,4-8,5-5
17,1610612757,Portland,Trail Blazers,West,8,11,9,8-11,3-5,5-6,3-7
19,1610612763,Memphis,Grizzlies,West,7,12,10,7-12,4-6,3-6,4-6


In [93]:
Teams('LAL').get_team_details()

{'id': 1610612747,
 'full_name': 'Los Angeles Lakers',
 'abbreviation': 'LAL',
 'nickname': 'Lakers',
 'city': 'Los Angeles',
 'state': 'California',
 'year_founded': 1948,
 'conference': 'West'}

In [27]:
eastern_conference = {
    'ATL', 'BOS', 'BKN', 'CHA', 'CHI', 'CLE', 'DET', 'IND',
    'MIA', 'MIL', 'NYK', 'ORL', 'PHI', 'TOR', 'WAS'
}

western_conference = {
    'DAL', 'DEN', 'GSW', 'HOU', 'LAC', 'LAL', 'MEM', 'MIN',
    'NOP', 'OKC', 'PHX', 'POR', 'SAC', 'SAS', 'UTA'
}

def check_valid_season(season: str = get_current_season()) -> str:
    """Check if the provided season is valid and return formatted season."""
    if not season:
        season = get_current_season()
    elif not season.isdigit():
        raise ValueError(f"Invalid season format: {season}. Expected a year in 'YYYY' format.")
    elif int(season) < 1980 or int(season) > datetime.now().year:
        raise ValueError(f"Invalid season year: {season}. Year must be between 1980 and {datetime.now().year}.")
    else:
        season = f"{season}-{str(int(season)+1)[-2:]}"
    
    return season

def get_all_teams()-> pd.DataFrame:
    try:
        all_teams = teams.get_teams()
        teams_df = pd.DataFrame(all_teams)
        return teams_df #.to_json(orient="records")
    except Exception as e:
        print(f"Error retrieving all teams: {e}")
        raise e

def get_team_details_by_abbreviation( team_abbreviation:str) -> Dict:
    try:
        team_details = teams.find_team_by_abbreviation(team_abbreviation)
        team_details["conference"] = "East" if team_details["abbreviation"] in eastern_conference else "West"
        return team_details
    except Exception as e:
        print(f"Error retrieving team details for abbreviation {team_abbreviation}: {e}")
        raise e

def get_team_details_by_id( team_id:int) -> Dict:
    try:
        team_details = teams.find_team_name_by_id(team_id)
        team_details["conference"] = "East" if team_details["abbreviation"] in eastern_conference else "West"
        return team_details
    except Exception as e:
        print(f"Error retrieving team details for ID {team_id}: {e}")
        raise e
    
def get_team_details_by_name( team_name:str) -> Dict:
    try:
        team_details = teams.find_teams_by_full_name(team_name)[0]
        team_details["conference"] = "East" if team_details["abbreviation"] in eastern_conference else "West"
        return team_details
    except Exception as e:
        print(f"Error retrieving team details for name {team_name}: {e}")
        raise e

def get_team_roster_per_season(team_id:int = 1610612747, season:str = None) -> pd.DataFrame:
    try:
        season = check_valid_season(season)

        team_id = get_team_details_by_id(team_id)["id"]
        roster = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
        roster_data = roster.get_data_frames()[0]
        roster_data = roster_data[["PLAYER", "NUM", "POSITION", "HEIGHT", "WEIGHT", "BIRTH_DATE", "AGE", "EXP", "SCHOOL"]]
        return roster_data
    except Exception as e:
        print(f"Error retrieving roster for team {team_id} in season {season}: {e}")
        raise e

def get_team_last_n_games_played(team_id:int = 1610612747, season:str = None, last_n_games:int = 5) -> pd.DataFrame:
    season = check_valid_season(season)
    team_details = get_team_details_by_id(team_id)
    team_id = team_details["id"]
    game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)
    game_log = game_log.get_data_frames()[0]

    game_log = game_log[['Game_ID', 'GAME_DATE', 'MATCHUP', 'WL']]
    return game_log[:last_n_games]

def get_team_league_standing(team_id:int = 1610612747, season:str = None) -> pd.DataFrame:
    season = check_valid_season(season)
    team_details = get_team_details_by_id(team_id)
    standings = get_current_standings(conference=team_details['conference']).reset_index()
    index = int(standings[standings['TeamName'] == team_details['nickname']].index[0]) + 1

    return index


def get_team_full_info( season:str = None):
    pass



        
    

In [28]:
#get_all_teams()
get_team_roster_per_season(team_id=1610612747, season=None)

,PLAYER,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL
0,Adou Thiero,1,G,6-7,220,"MAY 08, 2004",21.0,R,Arkansas
1,Jarred Vanderbilt,2,F,6-8,214,"APR 03, 1999",26.0,7,Kentucky
2,Dalton Knecht,4,F,6-6,215,"APR 19, 2001",24.0,1,Tennessee
3,Deandre Ayton,5,C,7-0,252,"JUL 23, 1998",27.0,7,Arizona
4,Gabe Vincent,7,G,6-2,200,"JUN 14, 1996",29.0,6,California-Santa Barbara
5,Bronny James,9,G,6-2,210,"OCT 06, 2004",21.0,1,Southern California
6,Jaxson Hayes,11,C-F,7-0,220,"MAY 23, 2000",25.0,6,Texas
7,Jake LaRavia,12,F,6-7,235,"NOV 03, 2001",24.0,3,Wake Forest
8,Maxi Kleber,14,F,6-10,240,"JAN 29, 1992",33.0,8,Bayern Munich
9,Austin Reaves,15,G,6-5,197,"MAY 29, 1998",27.0,4,Oklahoma


In [29]:
get_team_details_by_id(1610612747)

{'id': 1610612747,
 'full_name': 'Los Angeles Lakers',
 'abbreviation': 'LAL',
 'nickname': 'Lakers',
 'city': 'Los Angeles',
 'state': 'California',
 'year_founded': 1948,
 'conference': 'West'}

In [26]:
get_team_league_standing(team_id=1610612747, season=None)

4

In [ ]:
teams_df = get_all_teams()
teams_df['conference'] = teams_df['abbreviation'].apply(lambda abb: "East" if abb in eastern_conference else "West")
teams_df.drop()
teams_df

,id,full_name,abbreviation,nickname,city,state,year_founded,conference
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949,East
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946,East
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970,East
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002,West
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966,East
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980,West
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976,West
7,1610612744,Golden State Warriors,GSW,Warriors,San Francisco,California,1946,West
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967,West
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970,West
